In [ ]:
import data_io as io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sp
import os

label_file = r'/home/Eric/Documents/gitRepos/cv4e_dataPrep/fx_labels_Bp.csv'
data_path = r'/mnt/class_data/esnyder/raw_data'
labels = pd.read_csv(label_file)

# Image parameters
twin_s = 2
x_span_pxl = 1250
x_overlap_pxl = int(x_span_pxl * 0.5)

# Pick a random label row
row = labels.iloc[200]
data_filepath = os.path.join(data_path, row['dataset'], row['source_file'])

# Find all labels in same dataset & time
same_labels = labels[
    (labels['source_file'] == row['source_file']) &
    (labels['t'] == row['t'])
]

# Load corresponding dataset + settings
settings_file = io.find_settings_h5(data_filepath)
settings = io.load_settings_preprocessed_h5(settings_file)

nonzeros = settings['rehydration_info']['nonzeros_mask']
original_shape = settings['rehydration_info']['target_shape']

fk_dehyd, timestamp = io.load_preprocessed_h5(data_filepath)
tx = 1e9 * io.rehydrate(fk_dehyd, nonzeros, original_shape, return_format='tx')

fs = settings['processing_settings']['fs']
dx = settings['processing_settings']['dx']

# Filter
b, a = sp.butter(10, 70 / (0.5 * fs), btype='low')
tx = sp.filtfilt(b, a, tx, axis=1)

# Define time window
num_samples = int(row['win_length_s'] * fs)
t_idx_start = int(row['t'] * fs)

# Extract TX segment & compute FFT
tx_seg = tx[:, t_idx_start:t_idx_start + num_samples]
fx_fullCable = np.abs(np.fft.rfft(tx_seg, axis=1))

# Plot full cable FX image
fig_w = fx_fullCable.shape[1] // 10  # width in inches (freq bins)
fig_h = fx_fullCable.shape[0] // 10 # height in inches (spatial samples)

fig = plt.figure(figsize=(fig_w, fig_h))
plt.imshow(fx_fullCable, aspect='equal', origin='lower', vmin=0, vmax=30,
           extent=[0, fs/2, 0, dx*fig_h])

# Draw bounding boxes for all matching labels
for _, lab in same_labels.iterrows():
    # Pixel coords
    # pix_xmin = int(lab['f_min_hz'] * fs )
    # pix_xmax = int(lab['f_max_hz'] * fs )
    # pix_ymin = int(lab['x_min_m'] / dx)
    # pix_ymax = int(lab['x_max_m'] / dx)
    print('f_min={}, f_max={}, x_min={}, x_max={}'.format(
        lab['f_min_hz'], lab['f_max_hz'], lab['x_min_m'], lab['x_max_m']
        ))
    f_num_pxl = fig_w
    x_num_pxl = fig_h
    
    data_fmax_hz = fs / 2
    data_xmax_m = dx * x_num_pxl

    pix_xmin = int(lab['f_min_hz']  * f_num_pxl/data_fmax_hz)
    pix_xmax = int(lab['f_max_hz'] *  f_num_pxl/data_fmax_hz)
    pix_ymin = int(lab['x_min_m']  * x_num_pxl/data_xmax_m)
    pix_ymax = int(lab['x_max_m']  * x_num_pxl/data_xmax_m)

    rect = plt.Rectangle(
        (pix_xmin, pix_ymin),
        pix_xmax - pix_xmin,
        pix_ymax - pix_ymin,
        linewidth=2, edgecolor='r', facecolor='none'
    )
    plt.gca().add_patch(rect)



# No axes, save
# plt.axis('off')
# plt.tight_layout()
fig.savefig('test_fx_full_boxes.png')
plt.show()
# plt.close(fig)

print(f"Saved image: test_fx_full_boxes.png")
print(f"Labels plotted: {len(same_labels)} from file {row['source_file']} at t={row['t']} seconds.")

In [ ]:
import data_io as io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sp
import os

label_file = r'/home/Eric/Documents/gitRepos/cv4e_dataPrep/fx_labels_Bp.csv'
data_path = r'/mnt/class_data/esnyder/raw_data'
labels = pd.read_csv(label_file)

# Image parameters
twin_s = 2
x_span_pxl = 1250
x_overlap_pxl = int(x_span_pxl * 0.5)

# Pick a random label row
row = labels.iloc[200]
data_filepath = os.path.join(data_path, row['dataset'], row['source_file'])

# Find all labels in same dataset & time
same_labels = labels[
    (labels['source_file'] == row['source_file']) &
    (labels['t'] == row['t'])
]

# Load corresponding dataset + settings
settings_file = io.find_settings_h5(data_filepath)
settings = io.load_settings_preprocessed_h5(settings_file)

nonzeros = settings['rehydration_info']['nonzeros_mask']
original_shape = settings['rehydration_info']['target_shape']

fk_dehyd, timestamp = io.load_preprocessed_h5(data_filepath)
tx = 1e9 * io.rehydrate(fk_dehyd, nonzeros, original_shape, return_format='tx')

fs = settings['processing_settings']['fs']
dx = settings['processing_settings']['dx']

# Filter
b, a = sp.butter(10, 70 / (0.5 * fs), btype='low')
tx = sp.filtfilt(b, a, tx, axis=1)

# Define time window
num_samples = int(row['win_length_s'] * fs)
t_idx_start = int(row['t'] * fs)

# Extract TX segment & compute FFT
tx_seg = tx[:, t_idx_start:t_idx_start + num_samples]
fx_fullCable = np.abs(np.fft.rfft(tx_seg, axis=1))

# Plot full cable FX image
fig_w = fx_fullCable.shape[1] // 10  # width in inches (freq bins)
fig_h = fx_fullCable.shape[0] // 10 # height in inches (spatial samples)

fig = plt.figure(figsize=(fig_w, fig_h))
plt.imshow(fx_fullCable, aspect='equal', origin='lower', vmin=0, vmax=30,
           extent=[0, fs/2, 0, dx*fig_h])

# No axes, save
# plt.axis('off')
# plt.tight_layout()
fig.savefig('test_fx_full.png')
plt.show()
# plt.close(fig)

print(f"Saved image: test_fx_full.png")

In [ ]:
# Generate bounding boxes on top of the generated FX plot

# load image
image_path = 'test_fx_full.png'
image = plt.imread(image_path)

scale = 1/50

img_w = image.shape[1] * scale
img_h = image.shape[0] * scale

plt.figure(figsize=(img_w, img_h))
plt.imshow(image)

# bounding boxes
for _, lab in same_labels.iterrows():
    # Pixel coords
    # pix_xmin = int(lab['f_min_hz'] * fs )
    # pix_xmax = int(lab['f_max_hz'] * fs )
    # height, width = image.shape[0], image.shape[1]
    data_fmax_hz = fs / 2
    data_xmax_m = 46566 # dx * img_h


    pix_xmin = int(lab['f_min_hz']  * img_w/data_fmax_hz)
    pix_xmax = int(lab['f_max_hz'] *  img_w/data_fmax_hz)
    pix_ymin = int(lab['x_min_m']  * img_h/data_xmax_m)
    pix_ymax = int(lab['x_max_m']  * img_h/data_xmax_m)

    rect = plt.Rectangle(
        (pix_xmin, pix_ymin),
        pix_xmax - pix_xmin,
        pix_ymax - pix_ymin,
        linewidth=2, edgecolor='r', facecolor='none'
    )
    plt.gca().add_patch(rect)
plt.show()